In [2]:
import sys
from pathlib import Path

In [3]:
src_path = Path("../src").resolve()
sys.path.append(str(src_path))

In [6]:
from sqlmodel import Session
from api.db.session import engine
from api.events.models import EventModel

In [13]:
from sqlalchemy import select
from sqlalchemy.orm import Session

with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.updated_at.asc()).limit(10)
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query)
    print("")
    # results = session.exec(query).all()
    print(str(query))


SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT 10

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT :param_1


In [ ]:
from timescaledb.hyperfunctions import time_bucket 
from sqlalchemy import func
from pprint import pprint
from datetime import datetime, timedelta, timezone

with Session(engine) as session:
    bucket = time_bucket("1 day", EventModel.time)
    pages = ['/about', '/contact', '/pages', '/pricing']
    start = datetime.now(timezone.utc) - timedelta(hours=1)
    finish = datetime.now(timezone.utc) + timedelta(hours=1)
    query(
        select(
            bucket,
            EventModel.page,
            func.count()
        )
        .where(
            EventModel.time > start,
            EventModel.time <= finish,
            EventModel.page.in_(pages)
        )
        .group_by(
            bucket,
            EventModel.page
        )
        .order_by(
            bucket,
            EventModel.page
        )
    )
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    results = session.execute(query).fetchall()
    pprint(results)


[]
